##### First we import all libraries required for this notebook

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests
from bs4 import BeautifulSoup
# import k-means from clustering stage
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

##### Read the Toronto postal code data from wiki
##### Use the BeautifulSoup library to scrap the data

In [2]:
toronto_neigh  = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
toronto_neigh_text = toronto_neigh.text
toronto_neigh_soup = BeautifulSoup(toronto_neigh_text, "lxml")

##### Create dataframe with 3 columns using pandas. 

In [3]:
pd.options.display.max_rows = 999
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
# instantiate the dataframe
toronto_nhoods = pd.DataFrame(columns=column_names)

##### Scrap the data by reading the main table using its classname. 
##### Read all rows. If the row does not contain 3  td's then ignore this row as this row will not contain relevant data
##### Check if value of Borough is 'Not assigned'. If it is, then ignore this row.
##### If the value of Borough is 'Not assigned' but the value of Neighborhood is 'Not assigned', then assign value of Burrough to Neighborhood for this row.
##### FILL the dataframe with data for 3 columns

In [4]:
table_main = toronto_neigh_soup.find('table', class_='wikitable sortable')

tablebody_main = table_main.find('tbody')
#print(tablebody_main)
tablerows_all = tablebody_main.find_all('tr')

for row in tablerows_all:
    td_postalcode = row.find_all('td')
    
    if len(td_postalcode) == 3:
        if "Not assigned" not in td_postalcode[1]:
            
            if (td_postalcode[2].text.replace("\n","") == "Not assigned"):
                nName = td_postalcode[1].text
            else:
                nName = td_postalcode[2].text.replace("\n","")
                                
            toronto_nhoods = toronto_nhoods.append({'PostalCode': td_postalcode[0].text ,
                                                  'Borough': td_postalcode[1].text ,
                                                  'Neighborhood': nName,
                                                      }, ignore_index=True)
    
    
    

##### Verify the data

In [5]:
toronto_nhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


##### We want to group the data by Postalcode, Borough and then concatenate (join) Neighborhood column so that we get a unique row for each Postalcode/Borough 

In [6]:
toronto_nhood_grouped = toronto_nhoods.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()
toronto_nhood_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


##### Use the shape method to get the number of rows and columns in the dataframe
##### There are 103 rows and 3 columns

In [7]:
toronto_nhood_grouped.shape

(103, 3)

##### First tried using geocoder to get latitude and longitude. However the service was not returning results.
##### After that I received message for daily max limit on calls made to geocoder (This is shown in last lines of next cell) 

In [8]:
#!conda install -c conda-forge geocoder --yes
#import geocoder
#toronto_nhoods_LatLong = toronto_nhood_grouped
#HeaderList_LatLong = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', "Longitude"]
#toronto_nhoods_LatLong = toronto_nhoods_LatLong.reindex(columns = HeaderList_LatLong)  

#pc = 'M1E'
#g = geocoder.google('{}, Toronto, Ontario'.format(pc))
#lat_lng_coords = g.latlng

#for index, row in toronto_nhoods_LatLong.iterrows():
#    pc = row["PostalCode"]
#    lat_lng_coords = None
    
    #while(lat_lng_coords is None):
#    for i in range(1,20):
#        if (lat_lng_coords is None):
#            print(pc + '   '  + i + '    A')
#            g = geocoder.google('{}, Toronto, Ontario'.format(pc))
#            lat_lng_coords = g.latlng
#        else:
#            print(pc + '   '  + i + '   ' + lat_lng_coords)


#<[OVER_QUERY_LIMIT] Google - Geocode [empty]>
#None

##### Used alternative method to read the csv file. Uploaded the file to Watson Studio and read into a dataframe

In [9]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


##### Renamed column to match with main dataframe. Then merged the main dataframe with this new dataframe.
##### Now we have 1 dataframe with all rows and columns

In [10]:
df_data_latlong.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

In [11]:
df_data_latlong.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
toronto_nhood_full = pd.merge(toronto_nhood_grouped, df_data_latlong, on="PostalCode")

In [13]:
toronto_nhood_full.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
print('The dataframe has {} districts and {} neighborhoods.'.format(
        len(toronto_nhood_full['Borough'].unique()),
        toronto_nhood_full.shape[0]
    )
)

The dataframe has 11 districts and 103 neighborhoods.


##### We will only analyze, cluster, and segment part of the dataset
##### Filtered the data frame to get data where Borough contains word Toronto. 

In [15]:
toronto_data = toronto_nhood_full[toronto_nhood_full['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()
toronto_data.shape

(38, 5)

##### Credentials for FourSquare API

In [16]:
CLIENT_ID = '53LUJBTBAEFNV5HS3VTTJ5DP3WCODD4SBENMHE4MOCKR5TGK' # your Foursquare ID
CLIENT_SECRET = 'BJ0P4G2WIEOU5FKWHEUGCOARY2NZYXMPXIFWU5ZPVN5MSZJR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=200
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 53LUJBTBAEFNV5HS3VTTJ5DP3WCODD4SBENMHE4MOCKR5TGK
CLIENT_SECRET:BJ0P4G2WIEOU5FKWHEUGCOARY2NZYXMPXIFWU5ZPVN5MSZJR


##### Function to get Nearby Venues for each neighborhood

##### Made call to this function to get the venues df = toronto_venues

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name + '  ' + str(lat) + '  ' + str(lng))
        if not (str(lat) == '' or str(lng)==''):
            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                LIMIT)

            # make the GET request
            results = requests.get(url).json()["response"]['groups'][0]['items']
           
            # return only relevant information for each nearby venue
            venues_list.append([(                
                name, 
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [18]:
toronto_venues = None
toronto_venues = getNearbyVenues(toronto_data['Neighborhood'],toronto_data['Latitude'],toronto_data['Longitude'])

In [19]:
print(toronto_venues.shape)
#toronto_venues

(1697, 7)


##### Group dataframe by Neighborhood to get a count of venues for each neighborhood

In [20]:
toronto_venues.groupby(['Neighborhood']).count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,54,54,54,54,54,54
"Brockton,Exhibition Place,Parkdale Village",21,21,21,21,21,21
Business reply mail Processing Centre969 Eastern,18,18,18,18,18,18
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",14,14,14,14,14,14
"Cabbagetown,St. James Town",47,47,47,47,47,47
Central Bay Street,80,80,80,80,80,80
"Chinatown,Grange Park,Kensington Market",97,97,97,97,97,97
Christie,16,16,16,16,16,16


In [21]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.


## Now we will analyze neighborhood

##### Converted Venue Category to dummy variable

In [22]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
tornto_onehot = toronto_onehot[fixed_columns]

In [23]:
toronto_onehot.head()

,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
toronto_onehot.shape

(1697, 237)

##### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [25]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,...,0.0,0.0,0.01,0.0,0.0,0.01,0.0,0.0,0.01,0.000000
1,Berczy Park,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
3,Business reply mail Processing Centre969 Eastern,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.055556
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.0,0.0,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.00,...,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.000000


In [26]:
toronto_grouped.shape

(38, 237)

##### Print each neighborhood along with the top 5 most common venues

In [27]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3      Thai Restaurant  0.04
4  American Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.06
2          Restaurant  0.06
3  Italian Restaurant  0.04
4         Cheese Shop  0.04


----Brockton,Exhibition Place,Parkdale Village----
                venue  freq
0         Coffee Shop  0.14
1      Breakfast Spot  0.10
2                Café  0.10
3                 Gym  0.05
4  Falafel Restaurant  0.05


----Business reply mail Processing Centre969 Eastern----
                  venue  freq
0    Light Rail Station  0.11
1            Restaurant  0.06
2                   Spa  0.06
3               Brewery  0.06
4  Fast Food Restaurant  0.06


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0    Air

##### Function to sort venues in descending order

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

##### Lets analyze and see the TOP 10 venues for each neighborhood

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

In [30]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Thai Restaurant,American Restaurant,Steakhouse,Gym,Restaurant,Asian Restaurant,Clothing Store,Bakery
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Bakery,Steakhouse,Cheese Shop,Pub,Café,Italian Restaurant,Farmers Market
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Café,Breakfast Spot,Gym / Fitness Center,Italian Restaurant,Pet Store,Performing Arts Venue,Climbing Gym,Caribbean Restaurant,Burrito Place
3,Business reply mail Processing Centre969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Pizza Place,Park,Recording Studio,Restaurant,Butcher,Burrito Place,Brewery
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Sculpture Garden,Boutique,Boat or Ferry,Airport Gate,Plane,Airport
5,"Cabbagetown,St. James Town",Restaurant,Coffee Shop,Pizza Place,Café,Indian Restaurant,Chinese Restaurant,Pub,Italian Restaurant,Bakery,Park
6,Central Bay Street,Coffee Shop,Italian Restaurant,Ice Cream Shop,Café,Burger Joint,Sandwich Place,Bar,Bubble Tea Shop,Indian Restaurant,Falafel Restaurant
7,"Chinatown,Grange Park,Kensington Market",Café,Bar,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Burger Joint
8,Christie,Grocery Store,Café,Park,Italian Restaurant,Athletics & Sports,Baby Store,Restaurant,Nightclub,Diner,Coffee Shop
9,Church and Wellesley,Sushi Restaurant,Japanese Restaurant,Coffee Shop,Gay Bar,Burger Joint,Restaurant,Pub,Men's Store,Gastropub,Fast Food Restaurant


## Cluster Neighborhoods

In [31]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:60] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2,
       0, 4, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

##### Merge the dataframe with cluster labels created using the KMeans model

In [32]:
toronto_merged = toronto_data
# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Pub,Astrologer,Yoga Studio,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Furniture / Home Store,Pub,Pizza Place,Liquor Store
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0,Sandwich Place,Park,Food & Drink Shop,Sushi Restaurant,Italian Restaurant,Steakhouse,Movie Theater,Pub,Ice Cream Shop,Burrito Place
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Coworking Space,Seafood Restaurant,Sandwich Place
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Bus Line,Park,Swim School,Dim Sum Restaurant,Yoga Studio,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


##### Install and import Folium package for maps

In [33]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

##### Create the Toronto Map first using Folium
##### Add markers on the map using the merged dataframe
##### Visualize the clusters for Toronto neighborhood

In [34]:
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [35]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Coffee Shop,Pub,Astrologer,Yoga Studio,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
1,East Toronto,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Furniture / Home Store,Pub,Pizza Place,Liquor Store
2,East Toronto,0,Sandwich Place,Park,Food & Drink Shop,Sushi Restaurant,Italian Restaurant,Steakhouse,Movie Theater,Pub,Ice Cream Shop,Burrito Place
3,East Toronto,0,Café,Coffee Shop,Gastropub,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Coworking Space,Seafood Restaurant,Sandwich Place
4,Central Toronto,0,Bus Line,Park,Swim School,Dim Sum Restaurant,Yoga Studio,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
5,Central Toronto,0,Clothing Store,Breakfast Spot,Burger Joint,Food & Drink Shop,Park,Hotel,Sandwich Place,Grocery Store,Eastern European Restaurant,Electronics Store
6,Central Toronto,0,Sporting Goods Shop,Coffee Shop,Clothing Store,Yoga Studio,Fast Food Restaurant,Mexican Restaurant,Diner,Dessert Shop,Park,Chinese Restaurant
7,Central Toronto,0,Coffee Shop,Sandwich Place,Dessert Shop,Café,Seafood Restaurant,Sushi Restaurant,Italian Restaurant,Pizza Place,Chinese Restaurant,Gourmet Shop
8,Central Toronto,0,Playground,Summer Camp,Intersection,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
9,Central Toronto,0,Coffee Shop,Pub,American Restaurant,Convenience Store,Sports Bar,Bagel Shop,Supermarket,Sushi Restaurant,Fried Chicken Joint,Light Rail Station


In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Downtown Toronto,1,Coffee Shop,Italian Restaurant,Ice Cream Shop,Café,Burger Joint,Sandwich Place,Bar,Bubble Tea Shop,Indian Restaurant,Falafel Restaurant
27,Downtown Toronto,1,Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Sculpture Garden,Boutique,Boat or Ferry,Airport Gate,Plane,Airport


In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,2,Garden,Ice Cream Shop,Yoga Studio,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Downtown Toronto,3,Coffee Shop,Restaurant,Café,Seafood Restaurant,Cocktail Bar,Pub,Hotel,Italian Restaurant,Japanese Restaurant,Art Gallery


In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Central Toronto,4,Café,Coffee Shop,Sandwich Place,Pizza Place,Jewish Restaurant,BBQ Joint,French Restaurant,Indian Restaurant,Pub,Burger Joint
